In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

from warnings import simplefilter

2024-01-30 00:26:36.821190: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 00:26:36.825466: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 00:26:37.029488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 00:26:37.029546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 00:26:37.031554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

In [3]:
#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

Ready to generate train and test datasets


In [7]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Compute metrics
    precision = metrics.precision_score(y_test, y_pred_classes, average='weighted')
    accuracy = metrics.accuracy_score(y_test, y_pred_classes)
    f1 = metrics.f1_score(y_test, y_pred_classes, average='macro')
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Precision: {precision}\nAccuracy: {accuracy}\nF1 Score: {f1}\nAUC: {auc}")
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [5]:
timesteps = 1  # Number of timesteps; adjust based on your data
features_per_timestep = x_train.shape[1]  # Features per timestep

x_train_reshaped = x_train.reshape(-1, timesteps, features_per_timestep)
x_test_reshaped = x_test.reshape(-1, timesteps, features_per_timestep)

# Define the RNN model
output_shape = len(np.unique(y_train))


# Function to create the RNN model with an additional hidden layer
def create_rnn_model(units=32, activation='relu', hidden_units=32, optimizer='adam'):
    model = Sequential([
        SimpleRNN(units, activation=activation, input_shape=(timesteps, features_per_timestep)),
        Dense(units, activation=activation),  # Hidden dense layer
        Dense(output_shape, activation='softmax')  # Output layer
    ])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrapping the model with KerasClassifier
rnn_model = KerasClassifier(model=create_rnn_model, verbose=1, units = [32, 64, 128], batch_size = [32, 64, 128, 256, 512, 1024], epochs = [15, 20, 25])

# Define the parameter grid for hyperparameter tuning
param_dist = {
    'units': [32, 64, 128],
    'batch_size': [32, 64, 128, 256, 512, 1024],
    'epochs': [15, 20, 25]
}

# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rnn_model, param_distributions=param_dist, 
                                   n_iter=10, cv=3, verbose=2)

# Perform hyperparameter tuning using x_val and y_val
random_search_result = random_search.fit(x_train_reshaped, y_train)

# Best parameters
best_params = random_search_result.best_params_
print(f"Best Parameters: {best_params}")

# Train the optimal RNN model with best parameters
optimal_rnn_model = create_rnn_model(units=best_params['units'])
optimal_rnn_model.fit(x_train_reshaped, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/15
1207/1207 [==============================] - 27s 17ms/step - loss: 5.6587 - accuracy: 0.8136
Epoch 2/15
1207/1207 [==============================] - 19s 15ms/step - loss: 1.4670 - accuracy: 0.8739
Epoch 3/15
1207/1207 [==============================] - 20s 16ms/step - loss: 1.1102 - accuracy: 0.8956
Epoch 4/15
1207/1207 [==============================] - 18s 15ms/step - loss: 0.8473 - accuracy: 0.9033
Epoch 5/15
1207/1207 [==============================] - 17s 14ms/step - loss: 0.6494 - accuracy: 0.9089
Epoch 6/15
1207/1207 [==============================] - 18s 15ms/step - loss: 0.4834 - accuracy: 0.9144
Epoch 7/15
1207/1207 [==============================] - 17s 14ms/step - loss: 0.4191 - accuracy: 0.9200
Epoch 8/15
1207/1207 [==============================] - 17s 14ms/step - loss: 0.3512 - accuracy: 0.9246
Epoch 9/15
1207/1207 [==============================] - 16s 14ms/step - loss: 0.3093 - accuracy: 0.9242
Epo

2024-01-30 00:57:21.538846: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_3/simple_rnn_3/while/body/_1/sequential_3/simple_rnn_3/while/simple_rnn_cell/Relu}}
	.  Registered:  <no registered kernels>

2024-01-30 00:57:21.545581: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:135] tfg_optimizer{any(tfg-consolidate-attrs,tfg-toposort,tfg-shape-inference{graph-version=0},tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential_3/simple_rnn_3/while/body/_1/sequential_3/simple_rnn_3/while/simple_rnn_cell/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2024-01-30 00:57:21.626143: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] remapper failed: NOT_FOUND: Op type not registered '' in binary running on jupyter-z4shang-40ucsd-2eedu. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loadi

2413/2413 [==============================] - 9s 4ms/step
[CV] END ................batch_size=32, epochs=25, units=128; total time=11.3min
Epoch 1/25
4826/4826 [==============================] - 25s 5ms/step - loss: 2.8951 - accuracy: 0.8588
Epoch 2/25
4826/4826 [==============================] - 27s 6ms/step - loss: 0.3802 - accuracy: 0.9003
Epoch 3/25
4826/4826 [==============================] - 27s 6ms/step - loss: 0.2811 - accuracy: 0.9004
Epoch 4/25
4826/4826 [==============================] - 29s 6ms/step - loss: 0.2554 - accuracy: 0.9077
Epoch 5/25
4826/4826 [==============================] - 26s 5ms/step - loss: 0.2595 - accuracy: 0.9030
Epoch 6/25
4826/4826 [==============================] - 26s 5ms/step - loss: 0.2406 - accuracy: 0.9132
Epoch 7/25
4826/4826 [==============================] - 24s 5ms/step - loss: 0.2367 - accuracy: 0.9155
Epoch 8/25
4826/4826 [==============================] - 23s 5ms/step - loss: 0.2429 - accuracy: 0.9181
Epoch 9/25
4826/4826 [================

2024-01-30 01:22:55.722360: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_6/simple_rnn_6/while/body/_1/sequential_6/simple_rnn_6/while/simple_rnn_cell/Relu}}
	.  Registered:  <no registered kernels>

2024-01-30 01:22:55.729040: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:135] tfg_optimizer{any(tfg-consolidate-attrs,tfg-toposort,tfg-shape-inference{graph-version=0},tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential_6/simple_rnn_6/while/body/_1/sequential_6/simple_rnn_6/while/simple_rnn_cell/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2024-01-30 01:22:55.734336: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] remapper failed: NOT_FOUND: Op type not registered '' in binary running on jupyter-z4shang-40ucsd-2eedu. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loadi

2413/2413 [==============================] - 7s 3ms/step
[CV] END .................batch_size=32, epochs=15, units=64; total time= 5.5min
Epoch 1/15
4826/4826 [==============================] - 24s 4ms/step - loss: 4.3524 - accuracy: 0.8375
Epoch 2/15
4826/4826 [==============================] - 21s 4ms/step - loss: 0.7979 - accuracy: 0.8954
Epoch 3/15
4826/4826 [==============================] - 22s 5ms/step - loss: 0.4113 - accuracy: 0.8961
Epoch 4/15
4826/4826 [==============================] - 21s 4ms/step - loss: 0.2934 - accuracy: 0.9003
Epoch 5/15
4826/4826 [==============================] - 21s 4ms/step - loss: 0.2478 - accuracy: 0.9082
Epoch 6/15
4826/4826 [==============================] - 22s 5ms/step - loss: 0.2330 - accuracy: 0.9160
Epoch 7/15
4826/4826 [==============================] - 22s 4ms/step - loss: 0.2388 - accuracy: 0.9133
Epoch 8/15
4826/4826 [==============================] - 22s 4ms/step - loss: 0.2346 - accuracy: 0.9128
Epoch 9/15
4826/4826 [================

2024-01-30 01:51:23.732674: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_18/simple_rnn_18/while/body/_1/sequential_18/simple_rnn_18/while/simple_rnn_cell/Relu}}
	.  Registered:  <no registered kernels>

2024-01-30 01:51:23.738762: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:135] tfg_optimizer{any(tfg-consolidate-attrs,tfg-toposort,tfg-shape-inference{graph-version=0},tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential_18/simple_rnn_18/while/body/_1/sequential_18/simple_rnn_18/while/simple_rnn_cell/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2024-01-30 01:51:23.743839: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] remapper failed: NOT_FOUND: Op type not registered '' in binary running on jupyter-z4shang-40ucsd-2eedu. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you a

2413/2413 [==============================] - 6s 2ms/step
[CV] END .................batch_size=32, epochs=25, units=64; total time= 8.7min
Epoch 1/25
4826/4826 [==============================] - 19s 3ms/step - loss: 3.8606 - accuracy: 0.8319
Epoch 2/25
4826/4826 [==============================] - 17s 3ms/step - loss: 0.8507 - accuracy: 0.8960
Epoch 3/25
4826/4826 [==============================] - 17s 3ms/step - loss: 0.3660 - accuracy: 0.9102
Epoch 4/25
4826/4826 [==============================] - 17s 3ms/step - loss: 0.2604 - accuracy: 0.9128
Epoch 5/25
4826/4826 [==============================] - 16s 3ms/step - loss: 0.2341 - accuracy: 0.9221
Epoch 6/25
4826/4826 [==============================] - 17s 3ms/step - loss: 0.2101 - accuracy: 0.9285
Epoch 7/25
4826/4826 [==============================] - 17s 3ms/step - loss: 0.2083 - accuracy: 0.9275
Epoch 8/25
4826/4826 [==============================] - 17s 3ms/step - loss: 0.1958 - accuracy: 0.9303
Epoch 9/25
4826/4826 [================

In [8]:
calculate_performance_metrics(x_test_reshaped, y_test, optimal_rnn_model)

3103/3103 [==============================] - 22s 7ms/step
Precision: 0.901511375773276
Accuracy: 0.8992647799375566
F1 Score: 0.7890605587467102
AUC: 0.977944504047672
Mean FNR: 0.24386009678008888
Mean TNR: 0.9724974745994196
Mean FPR: 0.02750252540058039
Mean TPR: 0.7561399032199111
